#### Prerequisites

In [ ]:
!pip install psycopg2-binary

In [ ]:
!pip install Faker

In [1]:
import pandas as pd

In [2]:
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [6]:
from faker import Faker

#### Establish Connection and create DB

In [19]:
try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="postgres",
                                  port="5432",
                                 database="postgres")

    cursor = connection.cursor()
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    print( "Connected to Postgres" )
except (Exception, Error) as error:
    print("Connection error to Postgres ", error)

Connected to Postgres


In [20]:
try: 
    cursor.execute("CREATE DATABASE exercise_db");
    print("Query executed successfully")
except(Exception, Error):
    print("Unable to create DB")

Unable to create DB


#### Common function to execute SQL statements

In [21]:
def execute_sttmt( sql_sttmt ):
    try: 
        cursor.execute(sql_sttmt);
        print("Query executed successfully")
    except (Error) as error:
        print("unable to execute query:" , sql_sttmt)
        print(error)

In [29]:
#execute_sttmt("drop table patient_scores")

Query executed successfully


#### Create exercise table

In [30]:
create_table_sttmt = '''CREATE TABLE IF NOT EXISTS patient_scores(
    id SERIAL,
    patient_id INT NOT NULL,
    scores JSON NOT NULL,
    date DATE
);'''

execute_sttmt(create_table_sttmt)

Query executed successfully


In [24]:
insert_entry_sttmt = '''INSERT INTO patient_scores (patient_id, scores, date) VALUES ({},{},{})'''

In [36]:
"""
Exercise default entries
"""
entries = [
    [1323, '{"satisfaction": 9, "pain": 2, "fatigue": 2}', "2020-06-25"],
    [9032, '{"satisfaction": 2, "pain": 7, "fatigue": 5}',"2020-06-30"],
    [2331, '{"satisfaction": 7, "pain": 1, "fatigue": 1}',"2020-07-05"],
    [2303, '{"satisfaction": 8, "pain": 9, "fatigue": 0}', "2020-07-12"],
    [1323, '{"satisfaction": 10, "pain": 0, "fatigue": 0}' ,"2020-07-09"],
    [2331, '{"satisfaction": 8, "pain": 9, "fatigue": 5}',"2020-07-20"]
]

In [37]:
"""
Generate random values with satisfaction starting at 6, 
just so everyone is not a detractor
"""
entries = []

for _ in range(100):
    fake_entry = [Faker().unique.random_int(), '{"satisfaction":'+str(Faker().unique.random_int(min=6, max=10, step=1))+' , "pain": '+str(Faker().unique.random_int(min=1, max=10, step=1))+', "fatigue": '+str(Faker().unique.random_int(min=1, max=10, step=1))+'}', Faker().date() ]
    entries.append(fake_entry)
    
len(entries)    

100

In [38]:
for entry in entries:
    execute_sttmt(insert_entry_sttmt.format( entry[0], "'" + entry[1] +"'" , "'" +entry[2]+"'" ))

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed succe

#### Create SQL query for NPS score
(with pandas)

In [33]:
# check info on newly created table
pd.read_sql_query(''' SELECT * FROM patient_scores''', connection)

,id,patient_id,scores,date
0,1,4576,"{'satisfaction': 9, 'pain': 9, 'fatigue': 3}",1989-09-20
1,2,1545,"{'satisfaction': 8, 'pain': 1, 'fatigue': 9}",1994-12-26
2,3,4251,"{'satisfaction': 10, 'pain': 1, 'fatigue': 10}",2015-03-24
3,4,4302,"{'satisfaction': 10, 'pain': 5, 'fatigue': 5}",2017-09-15
4,5,3476,"{'satisfaction': 10, 'pain': 3, 'fatigue': 2}",1971-03-31
...,...,...,...,...
95,96,8934,"{'satisfaction': 10, 'pain': 10, 'fatigue': 9}",2021-05-08
96,97,9159,"{'satisfaction': 10, 'pain': 2, 'fatigue': 8}",1991-10-14
97,98,521,"{'satisfaction': 6, 'pain': 10, 'fatigue': 5}",1981-04-14
98,99,6747,"{'satisfaction': 10, 'pain': 3, 'fatigue': 2}",1978-10-26


In [34]:
# let's see what each month looks like
pd.read_sql_query('''         
        SELECT (scores->>'satisfaction')::integer as satisfaction
            , (scores->>'satisfaction')::integer < 7 as is_detractor
            , (scores->>'satisfaction')::integer > 8 as is_promoter
            , to_char(date,'Month') as month_name 
        FROM 
            patient_scores
        ORDER BY 
            month_name ---just to see these together
''', connection)

,satisfaction,is_detractor,is_promoter,month_name
0,10,False,True,April
1,9,False,True,April
2,8,False,False,April
3,6,True,False,April
4,8,False,False,April
...,...,...,...,...
95,8,False,False,September
96,9,False,True,September
97,9,False,True,September
98,10,False,True,September


In [39]:
# execute aggregation to calculate NPS according to provided formula
pd.read_sql_query('''
    WITH calc_detract as (
        SELECT (scores->>'satisfaction')::integer as satisfaction
            , (scores->>'satisfaction')::integer < 7 as is_detractor
            , (scores->>'satisfaction')::integer > 8 as is_promoter
            , to_char(date,'Month') as month_name 
        FROM 
            patient_scores
    ) 
    SELECT 
            month_name
            --,COUNT(is_detractor OR NULL) AS detractors
            --,COUNT(is_promoter OR NULL) AS promoters
            --,COUNT(*) AS month_users
            ,(COUNT(is_promoter OR NULL)-COUNT(is_detractor OR NULL))::numeric(2) / COUNT(1)::numeric(2) * 100 as NPS
        FROM calc_detract
        GROUP BY month_name
''', connection)

,month_name,nps
0,February,9.090909
1,November,25.000000
2,September,41.176471
3,August,39.130435
4,October,36.842105
5,December,-16.666667
6,June,27.777778
7,March,47.619048
8,July,-6.250000
9,January,-16.666667


#### Close SQL connection

In [ ]:
# close postgres connection
if connection:
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")